In [1]:
from __future__ import print_function
import os
import sys
import cv2
from matplotlib import pyplot as plt
import numpy as np

from local_config import CURRENT_DIR
sys.path.append(os.path.join(CURRENT_DIR, ".."))

try:
    from config import Config
    from data_loader import WaymoStixelDataset
except:
    print("failed to load module")
    
from albumentations import (
    Resize,
    Compose,
    CLAHE,
    HueSaturationValue,
    RandomBrightness,
    RandomContrast,
    RandomGamma,
    ToFloat,
    Normalize,
    GaussNoise,
    RandomShadow,
    RandomRain,
)

import utility
import importlib
from models.stixel_net import build_stixel_net_inceptionV3

# TensorFlow
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import (
    ModelCheckpoint,
    ReduceLROnPlateau,
    EarlyStopping,
)

In [2]:
dt_config = Config()
dt_config.display()


Configurations:
CURRENT_DIR                    /pfs/data5/home/es/es_es/es_menzweil/StixelNet/obstacle_detection_stixelnet
DATA_PATH                      /pfs/data5/home/es/es_es/es_menzweil/StixelNet/obstacle_detection_stixelnet/data/StixelNet_Waymo
GROUND_TRUTH_PATH              /pfs/data5/home/es/es_es/es_menzweil/StixelNet/obstacle_detection_stixelnet/data/StixelNet_Waymo/waymo_train.txt
NUM_EPOCHS                     50
SAVED_MODELS_PATH              saved_models




In [3]:
## Train and Val set


train_aug = Compose(
    [
        GaussNoise(p=1.0),
        RandomShadow(p=0.5),
        RandomRain(p=0.5, rain_type="drizzle"),
        RandomContrast(limit=0.2, p=0.5),
        RandomGamma(gamma_limit=(80, 120), p=0.5),
        RandomBrightness(limit=0.2, p=0.5),
        HueSaturationValue(
            hue_shift_limit=5, sat_shift_limit=20, val_shift_limit=10, p=0.5
        ),
        CLAHE(p=0.5, clip_limit=2.0),
        Normalize(p=1.0),
    ]
)
    
# AUGMENTATIONS DISABLED
train_set = WaymoStixelDataset(
        data_path=dt_config.DATA_PATH,
        ground_truth_path=os.path.join(dt_config.DATA_PATH, "waymo_train.txt"),
        batch_size=8,
        transform=None,
        customized_transform=utility.HorizontalFlip(p=0.5),
    )




val_aug = Compose([Normalize(p=1.0)])
val_set = WaymoStixelDataset(
        data_path=dt_config.DATA_PATH,
        ground_truth_path=os.path.join(dt_config.DATA_PATH, "waymo_val.txt"),
        transform=None,
    )


/home/es/es_es/es_menzweil/anaconda3/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1826: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/es/es_es/es_menzweil/anaconda3/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1800: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


In [4]:
# StixelNet regression setup


model = build_stixel_net_inceptionV3()
opt = optimizers.Adam(0.001)
lossF = losses.MeanSquaredError()

model.compile(loss=lossF, optimizer=opt)
model.summary()

2022-03-11 20:06:49.960989: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-11 20:06:57.216693: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30980 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:b3:00.0, compute capability: 7.0

KeyboardInterrupt



In [ ]:
X,y = train_set[0]
print(np.shape(X))
print(np.shape(y))

In [ ]:
# Training

num_epoch = 1000

callbacks = [
    ModelCheckpoint(
        os.path.join(dt_config.SAVED_MODELS_PATH, "model-{epoch:03d}-{loss:.4f}.h5"),
        monitor="val_loss",
        verbose=1,
        save_best_only=False,
        mode="auto",
        save_freq="epoch",
    ),    
    EarlyStopping(
        monitor="val_loss", min_delta=0, patience=10, verbose=0, mode="auto"
    ),
]


history = model.fit(
        train_set,       
        validation_data=val_set,      
        epochs=num_epoch,
        callbacks=callbacks,
        shuffle=True,
    )